# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [2]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [3]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the Formula 1 World Championship in 2010. He was driving for the Red Bull Racing team.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [4]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel. 
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [5]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [8]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [9]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [10]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [11]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [12]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [13]:
# Zero shot prompt
context_user = [
    {'role':'system', 'content':
     """You are an expert on computer building like Linus Tech Tips and your job is to answer questions for users on an online forum.
     """}
]
print(return_OAIResponse("Should I let my fans run at minimum or automatic or at maximum, whats better for lower temperatures?", context_user))

Running your fans at maximum speed constantly may provide the lowest temperatures, but it may also be quite loud and put unnecessary wear and tear on the fans. 

For optimal cooling with lower noise levels, I would recommend setting your fans to run automatically based on temperature. This way, the fans will increase their speed when the system is under heavy load and generating more heat, and decrease their speed when the system is idle or running cooler. 

You can usually set up fan curves in your computer's BIOS or through third-party software to control how the fans speed up or slow down based on the temperature readings. This way, you can strike a balance between keeping your system cool and keeping noise levels down. 

It's also important to make sure your case has good airflow with intake and exhaust fans positioned properly to efficiently move air through the system.


In [14]:
print(return_OAIResponse("Help! My pc wont boot into windows what i can do to fix it?", context_user))

If your PC is not booting into Windows, there could be a few potential issues causing this problem. Here are several steps you can take to troubleshoot and potentially fix the issue:

1. **Check the BIOS settings**: Make sure that your BIOS is configured to boot from the correct drive where Windows is installed. You may need to enter the BIOS setup by pressing a specific key (usually F2, F10, or DEL) during the boot process. From there, navigate to the boot options and ensure that the correct drive is selected as the primary boot device.

2. **Check the hard drive connections**: Make sure that all the cables connecting your hard drive to the motherboard are securely attached. If you have multiple hard drives, ensure that the one with Windows installed is properly connected.

3. **Check for external devices**: Sometimes, external devices like USB drives or external hard drives can interfere with the boot process. Try disconnecting all external devices except for the essential ones (keyb

In [15]:
# Few shot prompt
context_user = [
    {'role':'system', 'content':
     """You are a mexican parent teaching, creating questions to simulate a math test.
     Question 1: 4+5
     Answer: 9

     Question 2: 10 x 3
     Answer: 30

    Question 3: 8 / 2
    Answer: 4
     """}
]
print(return_OAIResponse("Please come up with 10 questions", context_user))

Certainly! Here are 10 math questions for your test:

Question 1: 7 + 3
Answer: 10

Question 2: 12 - 6
Answer: 6

Question 3: 5 x 4
Answer: 20

Question 4: 18 / 3
Answer: 6

Question 5: 9 + 8
Answer: 17

Question 6: 25 - 13
Answer: 12

Question 7: 6 x 7
Answer: 42

Question 8: 36 / 4
Answer: 9

Question 9: 11 + 15
Answer: 26

Question 10: 50 - 27
Answer: 23

Feel free to use these questions for your math test!


# Results and Findings
The experiments conducted with the SQL-based bot yielded positive results. One of the most notable findings is that there were no hallucinations in the generated SQL queries throughout testing. This indicates that the prompt modifications and clear schema instructions have successfully guided the model to produce accurate and reliable SQL queries.

# Key Findings:
The model consistently generated correct SQL queries based on the provided schema.
Prompt adjustments, such as emphasizing key calculations (e.g., revenue = price * quantity), ensured that the model handled more complex queries effectively.
The absence of hallucinations shows that the SQL-based approach and prompt structure offer a robust solution for querying dynamic data without errors.
These results suggest that the current setup is highly reliable for retrieving accurate information from the database and can scale effectively as the database or queries grow in complexity.